# 문장 수준 임베딩
- 잠재 의미 분석(LSA) : 행렬분해
- 잠재 디리클레 할당(LDA) : 확률모델, Doc2Vecm ELMo, GPT, BERT
- 특히, GPT, BERT등은 Self Attention기반의 트랜스포머 네트워크로 구성됨

<b><h3>1. 잠재 의미 분석</h3></b><br> 
- 잠재 의미 분석 : 단어-문서 행렬이나 TF-IDF 행렬, 단어-문맥 행렬 또는 PMI행렬에 특이값 분해로 차원 축소를 시행하고 단어에 해당하는 벡터를 취해 임베딩
- 

<b><h3>3. 잠재 디리클레 할당(LDA)</h3></b><br> 

![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/lda%EB%8F%84%EC%8B%9D.png?raw=True)
- LDA는 토픽별 단어의 분포, 문서별 토픽의 분포를 추정
- 특정 토픽에 특정 단어가 나타날 확률, 위 도식에서 노란색 토픽엔 gene이라는 단어가 등장확률이 0.04, dna는 0.02, genetic은 0.01로 노란색 토픽은 대략 "유전자"라는 주제 예상
- 주어진 문서를 보면 노란색 토픽에 해당하는 단어가 많다. 따라서 문서의 메인 주제는 노란색 토픽일 가능성이 높다.
- LDA는 글쓰기 예를 자주 든다.
- 글을 쓰기 전에 (1) 주제를 정한다. (2) 실제 글을 쓸 때 어떤 단어를 쓸지 결정한다.
- LDA는 (1) 말뭉치로부터 얻은 토픽 분포로부터 토픽을 뽑는다. (2) 해당 토픽에 해당하는 단어를 뽑는다.
- LDA는 말뭉치 이면에 존재하는 정보를 추론해기에 잠재라는 이름을 붙인것이다.

![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA1.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA2.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA3.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA4.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA5.jpg?raw=True)
![LDA도식](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA6.jpg?raw=True)

# Topic Modeling, LDA 구현

In [1]:
### 변수 선언

from collections import Counter

# " n(d,k)"
# 각 토픽이 각 문서에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 문서를 의미
document_topic_counts = [Counter() for _ in documents]
# document_topic_counts[3][1] : 3번째 문서에서 토픽1과 관련있는 단어 수

# " V(k,w)"
# 각 단어가 각 토픽에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 토픽을 의미
topic_word_counts = [Counter() for _ in range(K)]
# topic_word_counts[2]['nlp'] : 토픽2에 nlp단어가 연관지어 등장한 횟수

# " 시그마 V(k,j)"
# 각 토픽에 할당되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)]

# " 시그마 n(d,i)"
# 각 문서에 포함되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 문서를 의미함
document_lengths = map(len, documents)  # map(a, b) b의 변수를 a적용하여 리스트

# 단어 종류의 수
distinct_word = set(word for document in documents for word in document)
V = len(distinct_words)

# 총 문서의 수
D = len(documents)

NameError: name 'documents' is not defined

In [ ]:
# d번째 문서 i번째 단어의 토픽 z(d,i)가 j번째 할당될 확률 = A X B 
# A : p_topic_given_document
# B : p_word_given_topic

def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) / (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)    
    return ((topic_word_counts[topic][word] + beta) / (topic_counts[topic] + V * beta))

def topic_weight(d, word ,k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [ ]:
# A X B 완료
# 깁스 샘플링을 통해 z(d,i)에 새로운 topic 할당

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

import random
def sample_from(weights):
    # i를 weights[i] / sum(weights)
    # 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd 
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

In [17]:
documents = [["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]]

In [ ]:
# inference에 필요한 기초 데이터 만듬
# 토픽수 K 등 하이퍼파라메터를 정하고, 각 단어를 임의의 토픽에 배정한 뒤 필요한 숫자 카운팅

random.seed(0)

# topic 수 지정
K=4

# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]

# 위와 같이 랜덤 초기화한 상태에서 
# AB를 구하는 데 필요한 숫자를 세어봄
for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

![TF-IDF행렬](https://github.com/tenjumh/Natural-Language-Processing/blob/master/KOREAN%20EMBEDDING/image/5%EC%9E%A5/LDA-3
.jpg?raw=True)